In [ ]:
# Librerias
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from fpdf import FPDF
import pdf_lib as pdf_lib

In [ ]:
# Lectura y limpieza de datos
datos=[]
ataque=pd.read_csv('data/attacking.csv').drop(['serial','match_played'],axis=1)
intentos=pd.read_csv('data/attempts.csv').drop(['serial','match_played'],axis=1)
defenza=pd.read_csv('data/defending.csv').drop(['serial','match_played'],axis=1)
pases=pd.read_csv('data/distributon.csv').drop(['serial','match_played'],axis=1)
porteros=pd.read_csv('data/goalkeeping.csv').drop(['serial','match_played'],axis=1)
goles=pd.read_csv('data/goals.csv').drop(['serial','match_played'],axis=1)
general=pd.read_csv('data/key_stats.csv').drop(['goals','assists'],axis=1)
datos=[intentos,defenza,pases,porteros,goles,general]
df=ataque
for i in datos:
    df=pd.merge(df,i,on=['player_name','club','position'],how='outer')
df.drop(df[df['player_name']=='Reinildo'].index,inplace=True)
df.drop(df[df['player_name']=='Burak Yilmaz'].index,inplace=True)
df.drop(df[df['player_name']=='Luis Díaz'].index,inplace=True)
df.drop(df[df['player_name']=='Brobbey'].index,inplace=True)
df['g_p']=df['goals']/df['match_played'] # Cantidad de goles por partidos jugados
df['g_m']=df['minutes_played']/df['goals'] # Cantidad de minutos para que marque un gol
df['a_p']=df['assists']/df['match_played'] # Cantidad de asistencias por partidos jugados
df['a_m']=df['minutes_played']/df['assists'] # Cantidad de minutos para que de una asistencia
df.insert(36,'no_penalties',df['goals']-df['penalties']) # Cantidad de goles que no fueron de penal
df.insert(9,'tiros_porteria',(100*df['on_target'])/df['total_attempts']) # Cantidad de tiros a porteria
df.insert(11,'tiros_no_porteria',(100*df['off_target'])/df['total_attempts']) # Cantidad de tiros sin direccion de porteria
df.insert(13,'tiros_bloqueados',(100*df['blocked'])/df['total_attempts']) # Cantidad de tiros bloqueados
df.insert(27,'salvadas_p',df['saved']/df['match_played']) # Cantidad de goles salvados por partido
df.insert(29,'goles_p',df['conceded']/df['match_played']) # Cantidad de goles permitidos por partido
df.insert(31,'penales_p',df['saved_penalties']/df['match_played']) # Cantidad de penales salvados por partido
df.insert(33,'cero_goles',(df['cleansheets']*100)/df['match_played']) # Cantidad de porterias en ceros por partido
df.fillna(0,inplace=True) # Relleno los valores NaN con 0
df.loc[df['total_attempts'] == 0.0, 'tiros_porteria'] = np.nan # Eliminar los valores 0 de tiros_porteria para que no se concideren en las graficas
df.loc[df['total_attempts'] == 0.0, 'tiros_no_porteria'] = np.nan # Eliminar los valores 0 de tiros_no_porteria para que no se concideren en las graficas
df.loc[df['total_attempts'] == 0.0, 'tiros_bloqueados'] = np.nan # Eliminar los valores 0 de tiros_bloqueados para que no se concideren en las graficas
df.loc[df['goals'] == 0.0, 'g_m'] = np.nan # Eliminar los valores 0 de g_m para que no se concideren en las graficas
df.loc[df['assists'] == 0.0, 'a_m'] = np.nan # Eliminar los valores 0 de a_m para que no se concideren en las graficas
equipos=pd.read_csv('data/club_matches.csv', encoding = 'cp1252').drop(['Unnamed: 0'],axis=1).set_index('club') # Leer partidos de equipos
equipos=pd.merge(equipos,df.groupby(['club'])['goals'].sum(), on='club') # Agregar goles totales por equipo
equipos['g_m']=equipos['goals']/equipos['teams_matches'] # Cantidad de goles por partido de equipos
df=pd.merge(df,equipos['teams_matches'],on='club') # Agregar cantidad de partidos por equipo a registro de jugadores

In [ ]:
# Generar graficas de jugador
jugador='De Gea'
# Seleccion de jugador
'''bandera=True
while bandera or df_j.shape[0]==0:
    bandera=False
    jugador=input('Ingrese el nombre del jugador ')
    df_j=df[df['player_name']==jugador].reset_index(drop=True)
    if df_j.shape[0]>1:
        print(df_j[['player_name','club']])
        indice_jugador=int(input(f'Existen {df_j.shape[0]} coincidencias.\nIngresa el numero del jugadar que quieres seleccionar'))
        df_j=df_j.loc[indice_jugador]'''

df_j=df[df['player_name']==jugador].reset_index(drop=True)
posicion=df_j['position'][0]
club=df_j['club'][0]
df_pos=df[df['position']==posicion]
df_club=df_pos[df_pos['club']==club]
dfs=[df_pos,df_club]


# Graficar
# Grafica Distribucion de goles
if df_j['goals'][0]>0: 
    # Datos a graficar
    colores1=cm.get_cmap('Set1', 4).colors
    colores2=cm.get_cmap('Dark2', 2).colors
    colores3=cm.get_cmap('tab10', 2).colors
    x1=df_j.loc[0,'right_foot':'others'][df_j.loc[0,'right_foot':'others']>0]
    x2=df_j.loc[0,'inside_area':'outside_areas'][df_j.loc[0,'inside_area':'outside_areas']>0]
    x3=df_j.loc[0,'penalties':'no_penalties'][df_j.loc[0,'penalties':'no_penalties']>0]
    plt.figure(figsize=(15,10)) # Crear grafica
    plt.subplot(111)
    # Crear graficas
    x1.plot.pie(legend=True,colors=colores1,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'k','fontsize':15},pctdistance=0.88)
    x2.plot.pie(legend=True,colors=colores2,radius=0.75,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'k','fontsize':15},pctdistance=0.88)
    x3.plot.pie(legend=True,colors=colores3,radius=0.5,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'k','fontsize':15},pctdistance=0.88)
    # Circulo central
    centre_circle=plt.Circle((0,0),0.25,fc='lightgray')
    fig=plt.gcf()
    fig.gca().add_artist(centre_circle)
    # Texto del circulo
    plt.text(-0.1,0.01,f"Goles: {int(df_j['goals'][0])}",fontdict={'fontweight':'bold','fontsize':15})
    plt.text(-0.15,-0.06,f"Partidos: {int(df_j['match_played'][0])}",fontdict={'fontweight':'bold','fontsize':15})
    plt.axis('equal') # Para que el circulo no tenga forma de elipse
    plt.legend(loc=7,fontsize=12) # Formato de la leyenda
    plt.ylabel('')
    plt.tight_layout()
    plt.savefig('media/plots/goles.png',bbox_inches='tight')
# Grafica Comparacion por posicion y Comparacion por posicion y equipo
for n in range(2):
    if posicion!='Goalkeeper':
        fig,ax=plt.subplots(nrows=5, ncols=3, figsize=(27,17)) # Crear figura
        columnas=[['goals','g_p','g_m'],['assists','a_p','a_m'],['tiros_porteria','tiros_no_porteria','tiros_bloqueados'],['pass_accuracy','cross_accuracy','offsides'],['corner_taken','freekicks_taken','dribbles']] # Columnas por fila a graficar
        titulos=[['Goles','Goles por partido','Minutos entre goles'],['Asistencias','Asistencias por partido','Minutos entre asistencias'],['% de tiros con direccion a porteria','% de tiros sin direccion a porteria','% de tiros bloqueados']\
                ,['Presicion de pase','Presicion de cambio de juego','Fuera de juego'],['Tiros de esquina','Tiros libres tirados','Regates']] # Titulos de las columna
        # Graficar
        for i,col in zip(range(5),columnas):
            for j in range(3): #datos    posicion    orientacion dar formato        ancho       colores->      caja                bordes caja                    max/min          ancho                          linea central     ancho                  tamaño circulos                       mediana
                dfs[n].boxplot([col[j]], ax=ax[i][j],vert=False, patch_artist=True, widths=0.25,boxprops=dict(facecolor='#A1C6EA', color='#558B6E'),capprops=dict(color='#558B6E', linewidth=3),whiskerprops=dict(color='#558B6E', linewidth=1.7),flierprops=dict(markersize=7),medianprops=dict(color='red'))
                ax[i][j].plot(df_j[col[j]][0],1, marker="D", color="#F4442E",markersize=10) # Datos jugador seleccionado/ figura diamante, color marcador, tamaño marcador
                ax[i][j].text(df_j[col[j]][0],0.75,f"{round(df_j[col[j]][0],2)}",fontdict={'fontsize':19,'color':'#F4442E','fontweight':'bold'},ha='center') # Texto en negritas 
                ax[i][j].tick_params(axis='x',labelsize=19,labelcolor='k')
                ax[i][j].tick_params(axis='y',labelsize=1)
                ax[i][j].set_facecolor('#E6E8E6') # Color de fondo
                ax[i][j].set_title(titulos[i][j],fontdict={'fontsize':30,'color':'#F18F01','fontweight':'bold'},pad=20) # Titulo de cada grafica
        ax[0][1].text(dfs[n]['g_p'].max(),1.3,f"Partidos jugados: {int(df_j['match_played'][0])}",fontdict={'fontsize':19,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto partidos jugados
        ax[0][2].text(dfs[n]['g_m'].max(),1.3,f"Minutos jugados: {int(df_j['minutes_played'][0])}",fontdict={'fontsize':19,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto minutos jugados
        ax[2][0].text(dfs[n]['tiros_porteria'].max(),1.3,f"Tiros totales: {int(df_j['total_attempts'][0])}",fontdict={'fontsize':19,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto tiros totales
        ax[3][0].text(dfs[n]['pass_accuracy'].max(),1.3,f"Pases totales: {int(df_j['pass_attempted'][0])}",fontdict={'fontsize':19,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto tiros totales
        ax[3][1].text(dfs[n]['cross_accuracy'].max(),1.3,f"Cambios de juego totales: {int(df_j['cross_attempted'][0])}",fontdict={'fontsize':19,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto tiros totales
        plt.tight_layout()
        plt.savefig(f'media/plots/comparacion{n}.png',bbox_inches='tight')
    else: # Graficas porteros
        fig,ax=plt.subplots(nrows=4, ncols=2, figsize=(21.6,11.4)) # Crear figura
        columnas=[['saved','salvadas_p'],['conceded','goles_p'],['saved_penalties','penales_p'],['cleansheets','cero_goles']] # Columnas por fila a graficar
        titulos=[['Tiros parados','Tiros parados por partido'],['Goles permitidos','Goles permitidos por partido'],['Penales salvados','Penales salvados por partido'],['Porterias en cero','Porcentaje de porterias en cero']] # Titulos de las columna
        # Graficar
        for i,col in zip(range(4),columnas):
            for j in range(2): #datos    posicion    orientacion dar formato        ancho       colores->      caja                bordes caja                    max/min          ancho                          linea central     ancho                  tamaño circulos                       mediana
                dfs[n].boxplot([col[j]], ax=ax[i][j],vert=False, patch_artist=True, widths=0.25,boxprops=dict(facecolor='#A1C6EA', color='#558B6E'),capprops=dict(color='#558B6E', linewidth=3),whiskerprops=dict(color='#558B6E', linewidth=1.7),flierprops=dict(markersize=7),medianprops=dict(color='red'))
                ax[i][j].plot(df_j[col[j]][0],1, marker="D", color="#F4442E",markersize=10) # Datos jugador seleccionado/ figura diamante, color marcador, tamaño marcador
                ax[i][j].text(df_j[col[j]][0],0.75,f"{round(df_j[col[j]][0],2)}",fontdict={'fontsize':19,'color':'#F4442E','fontweight':'bold'},ha='center') # Texto en negritas 
                ax[i][j].tick_params(axis='x',labelsize=19,labelcolor='k')
                ax[i][j].tick_params(axis='y',labelsize=1)
                ax[i][j].set_facecolor('#E6E8E6') # Color de fondo
                ax[i][j].set_title(titulos[i][j],fontdict={'fontsize':30,'color':'#F18F01','fontweight':'bold'},pad=20) # Titulo de cada grafica
        ax[0][1].text(dfs[n]['salvadas_p'].max(),1.3,f"Partidos jugados: {int(df_j['match_played'][0])}",fontdict={'fontsize':19,'color':'#02182B','fontweight':'semibold'},bbox={'facecolor':'#A0C1B9','alpha':0.3},ha='right') # Texto partidos jugados
        plt.tight_layout()
        plt.savefig(f'media/plots/comparacion{n}.png',bbox_inches='tight')
# Graficas minutos y partidos jugados
# Datos
p_j=df_j['match_played'][0] # Partidos jugados
p_e=df_j['teams_matches'][0] # Partidos posibles
m_j=df_j['minutes_played'][0] # Minutos jugados
m_p=df_j['teams_matches'][0]*90 # Minutos posibles
# Graficas
fig,ax=plt.subplots(nrows=2,ncols=1,figsize=(10,4)) # Crear subgrafica
plt.tight_layout() 
for i in range(2): # formato
    ax[i].spines.top.set_visible(False)
    ax[i].spines.right.set_visible(False)
    ax[i].spines.bottom.set_visible(False)
    ax[i].spines.left.set_visible(False)
    ax[i].tick_params(labelsize=18,labelcolor='k')
    ax[i].set_xticks([])
# Grafica partidos
ax[0].barh(['Partidos posibles','Partidos jugados'],[p_e,p_j],color=['#CC5803','#E2711D'],height=0.7) 
ax[0].text(p_j,1,f'{int(p_j)}',fontdict={'fontsize':20,'color':'k','fontweight':'bold'},ha='right') # Texto
ax[0].text(p_e,0.02,f'{int(p_e)}',fontdict={'fontsize':20,'color':'k','fontweight':'bold'},ha='right') # Texto
# Grafica minutos
ax[1].barh(['Minutos posibles','Minutos jugados'],[m_p,m_j],color=['#50858B','#62A8AC'],height=0.7) 
ax[1].text(m_j,1,f'{int(m_j)}',fontdict={'fontsize':20,'color':'k','fontweight':'bold'},ha='right') # Texto
ax[1].text(m_p,0.02,f'{int(m_p)}',fontdict={'fontsize':20,'color':'k','fontweight':'bold'},ha='right') # Texto
plt.savefig('media/plots/partidos_minutos.png',bbox_inches='tight')
    

In [ ]:
# Crear reporte
def reporte():
    tipo = True if posicion!='Goalkeeper' else False
    # Crear el documento PDF
    pdf=FPDF()
    pdf.set_auto_page_break(True,0)
    pdf=pdf_lib.portada(pdf,jugador,posicion,club) # Portada
    pdf=pdf_lib.introduccion(pdf) # Introduccion
    pdf=pdf_lib.pag_partidos(pdf) # Pagina partido y minutos
    if tipo:
        pdf=pdf_lib.pag_goles(pdf) # Pagina de goles
        pdf=pdf_lib.pag_comparacion0(pdf) # Comparaciones posicion y equipo
    else:
        pdf=pdf_lib.pag_comparacion1(pdf) # Comparaciones posicion y equipo
    pdf.output(f'/reportes/reporte_{jugador}.pdf','F')

reporte()